<a href="https://colab.research.google.com/github/coolhackboy/gpt-demo/blob/main/langchain_openai_Custom_Sales_Emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 项目描述
♈
介绍了如何使用 OpenAI 和 LangChain 自动地创建个性化的销售邮件。具体操作包括获取公司信息，使用自定义的 LangChain mapreduce prompts 和 prompt templates，以及最终生成一封定制的邮件。整个过程在 Y combinator startup directory 中进行演示，并提供了代码示例和讲解。

♈ 在行动建议部分，提到了需要提供一些公司信息，比如它们主要从事什么业务，以便让语言模型更好地理解和生成合适的内容。同时，建议不要滥发垃圾邮件，要认真对待每一封邮件。此外，作者还介绍了一些其他有用的工具，并建议大家关注他的 Twitter 账号以获取最新消息。

♈以上内容可以分为三个大部分，分别是
1/获取信息、
2/使用 LangChain prompts 和 prompt templates，
3/以及生成邮件。

♈通过具体案例和代码示例，详细讲解了如何操作。行动建议部分提供了实用的建议和注意事项。
本视频介绍的内容较为新颖，可以为需要进行自动化销售的企业或个人提供参考和思路。


【本项目描述由Chatgpt生成】

In [ ]:
!pip install langchain
!pip install unstructured
!pip install openai
!pip install tiktoken

In [5]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

In [ ]:
#@title openai_api_key
openai_api_key = '' #@param {type:"string"}
 
print(openai_api_key)

Load up your HTML from your company page


In [35]:
def get_company_page(company_path):

    y_combinator_url = f"https://www.ycombinator.com{company_path}"
    
    print (y_combinator_url)

    loader = UnstructuredURLLoader(urls=[y_combinator_url])
    return loader.load()


# Get the data of the company you're interested in
data = get_company_page("/companies/zapier")
    
print (f"You have {len(data)} document(s)")

https://www.ycombinator.com/companies/zapier
You have 1 document(s)


In [ ]:
print (f"Preview of your data:\n\n{data[0].page_content}")


In [30]:
# Split up the texts so you don't run into token limits
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 800,
    chunk_overlap  = 0
)

In [37]:
docs = text_splitter.split_documents(data)

print (f"You now have {len(docs)} documents")

You now have 3 documents


## Write your custom prompt templates
These will be used for your specific use case:

```map_prompt``` will be the prompt that is done on your first pass of your documents

```combine_prompt``` will be the prompt that is used when you combine the outputs of your map *pass*

In [18]:
map_prompt = """Below is a section of a website about {prospect}

Write a concise summary about {prospect}. If the information is not about {prospect}, exclude it from your summary.

{text}

CONCISE SUMMARY:"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text", "prospect"])



In [19]:
combine_prompt = """
Your goal is to write a personalized outbound email from {sales_rep}, a sales rep at {company} to {prospect}.

A good email is personalized and combines information about the two companies on how they can help each other.
Be sure to use value selling: A sales methodology that focuses on how your product or service will provide value to the customer instead of focusing on price or solution.

INFORMATION ABOUT {company}:
{company_information}

INFORMATION ABOUT {prospect}:
{text}

INCLUDE THE FOLLOWING PIECES IN YOUR RESPONSE:
- Start the email with the sentence: "We love that {prospect} helps teams..." then insert what they help teams do.
- The sentence: "We can help you do XYZ by ABC" Replace XYZ with what {prospect} does and ABC with what {company} does 
- A 1-2 setenece description about {company}, be brief
- End your email with a call-to-action such as asking them to set up time to talk more

YOUR RESPONSE:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["sales_rep", "company", "prospect", \
                                                                         "text", "company_information"])



In [20]:
company_information = """
* RapidRoad helps product teams build product faster
* We have a platform that allows product teams to talk more, exchange ideas, and listen to more customers
* Automated project tracking: RapidRoad could use machine learning algorithms to automatically track project progress, identify potential bottlenecks, and suggest ways to optimize workflows. This could help product teams stay on track and deliver faster results.
* Collaboration tools: RapidRoad could offer built-in collaboration tools, such as shared task lists, real-time messaging, and team calendars. This would make it easier for teams to communicate and work together, even if they are in different locations or time zones.
* Agile methodology support: RapidRoad could be specifically designed to support agile development methodologies, such as Scrum or Kanban. This could include features like sprint planning, backlog management, and burndown charts, which would help teams stay organized and focused on their goals.
"""


### LangChain Magic


In [67]:
llm = OpenAI(temperature=.7, openai_api_key=openai_api_key)

chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=map_prompt_template,
                             combine_prompt=combine_prompt_template,
                            #  verbose=True
                            )

In [63]:
output = chain({"input_documents": docs, # The seven docs that were created before
                "company": "RapidRoad", \
                "company_information" : company_information,
                "sales_rep" : "Greg", \
                "prospect" : "Zapier"
               })



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Below is a section of a website about Zapier

Write a concise summary about Zapier. If the information is not about Zapier, exclude it from your summary.

Accelerator

About

Apply

FAQ

People

YC Blog

Companies

Startup Directory

Top Companies

Founder Directory

Launch YC

Startup Jobs

All Jobs

Engineering

Operations

Marketing

Sales

Career Coaching

Pioneer Program 2023

Startup Job Guide

YC Startup Jobs Blog

Upcoming Events

Startup School

About

Co-Founder Matching

Startup School Blog

Library

SAFE

Resources

Event Calendar

Newsletter

For Investors

Hacker News

Open main menu

Apply for S2023 batch.

Apply

Home

Companies

Airbnb

Airbnb

Book accommodations around the world.

Y Combinator LogoW09

Public

Marketplace

Travel

Company

Jobs

News

http://airbnb.com

Book accommodations around the world.

CONCISE SUMMARY:
Prompt after formatting:
Below is a section of a website about Zapier

In [64]:
print (output['output_text'])



Dear Zapier Team,

We love that Zapier helps teams automate tasks and workflows, freeing up time and energy in managing repetitive tasks. We at RapidRoad believe we can help you take your automation even further. We can help you save time and energy by connecting apps and automating tasks with no coding knowledge by providing a platform that allows product teams to talk more, exchange ideas, and listen to more customers. RapidRoad's platform also offers automated project tracking, collaboration tools, and agile methodology support to help teams stay organized and focused on their goals. 

We believe RapidRoad will be a great addition to the Zapier platform and would love to discuss this further. Could we set up a call to discuss the potential partnership in more detail and how our automation platform can help your team?

We look forward to discussing this further.

Best,
Greg 
RapidRoad Sales Rep


### Rinse and Repeat: Loop Through Companies


In [43]:
import pandas as pd


In [65]:
df_companies = pd.DataFrame({
  'Name': ['Airbnb', 'DoorDash', 
           'Coinbase'],
  'Link': ['/companies/airbnb', '/companies/doordash', '/companies/coinbase'],
})

In [50]:
df_companies


,Naame,Link
0,Airbnb,/companies/airbnb
1,DoorDash,/companies/doordash
2,Coinbase,/companies/coinbase


In [68]:
for i, company in df_companies.iterrows():
    print (f"{i + 1}. {company['Name']}")
    page_data = get_company_page(company['Link'])
    docs = text_splitter.split_documents(page_data)

    output = chain({"input_documents": docs, \
                "company":"RapidRoad", \
                "sales_rep" : "Greg", \
                "prospect" : company['Name'],
                "company_information" : company_information
               })
    
    print (output['output_text'])
    print ("\n\n")

1. Airbnb
https://www.ycombinator.com/companies/airbnb

Dear Airbnb Team,

We love that Airbnb helps teams travel, explore, and experience the world in unique and meaningful ways. We believe that RapidRoad could help you take your mission to the next level.

RapidRoad is a platform that helps product teams build products faster. We have a suite of tools that help teams talk more, exchange ideas, and listen to customers better. We offer automated project tracking to help teams stay on track and deliver faster results. We also have built-in collaboration tools like shared task lists, real-time messaging, and team calendars. Furthermore, we are specifically designed to support agile development methodologies, such as Scrum or Kanban.

We believe that our platform could be a great addition to Airbnb and we would be honored to be a part of your mission. Let's set up a time to talk more about how RapidRoad can help you take your mission to the next level.

Thank you for your time and conside